# Naver blog scrap

## 라이브러리

In [883]:
import pandas as pd
import time, os, random
from datetime import datetime
from numpy import nan

In [884]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import chromedriver_autoinstaller as ca

## 크롤링 함수

In [885]:
def crawling(comment_areas, list):
    # 리스트 지정
    date_list = list[0]
    rare_number_list = list[1]
    check_reply_list = list[2]
    nickname_list = list[3]
    writer_url_list = list[4]
    like_num_list = list[5]
    link_num_list = list[6]
    reply_num_list = list[7]
    img_list = list[8]
    comment_list = list[9]

    for one in comment_areas:
        # 작성일 수집
        date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
        date_list.append(date)

        # 고유번호
        temp = one.attrs['class'][1].split('_')[-1]
        rare_number_list.append(temp)

        # 답글 구분
        try:
            one.find('span', 'u_cbox_secret_contents').text
            temp = '비밀댓글'
        except:
            try:
                temp = one.find('div', 'u_cbox_reply_area').find('li')
                temp = nan
            except:
                temp = '답글'
        check_reply_list.append(temp)

        # 닉네임
        try:
            temp = one.find('span', 'u_cbox_nick').text
        except:
            temp = '비밀댓글'
        nickname_list.append(temp)

        # 작성자 url
        try:
            temp = one.find('a', 'u_cbox_name').attrs['href']
        except:
            temp = '비밀댓글'
        writer_url_list.append(temp)

        # 공감수
        try:
            temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
        except:
            temp = 0
        like_num_list.append(temp)

        # 링크 수
        try:
            temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
            temp = len(temp)
        except:
            temp = 0
        link_num_list.append(temp)

        # 답글 수
        try:
            temp = one.find('div', 'u_cbox_reply_area').find_all('li')
            temp = len(temp)
        except:
            temp = 0
        reply_num_list.append(temp)

        # 첨부이미지url
        try:
            temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
        except:
            temp = nan
        img_list.append(temp)

        # 댓글 내용 수집
        try:
            temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
        except:
            temp = '비밀댓글'
        comment_list.append(temp)

    # 수집한 데이터 확인
    print('작성일 리스트 길이 : ', len(date_list))
    print('고유번호 리스트 길이 : ', len(rare_number_list))
    print('답글구분 리스트 길이 : ', len(check_reply_list))
    print('닉네임 리스트 길이 : ', len(nickname_list))
    print('작성자URL 리스트 길이 : ', len(writer_url_list))
    print('공감수 리스트 길이 : ', len(like_num_list))
    print('링크수 리스트 길이 : ', len(link_num_list))
    print('답글수 리스트 길이 : ', len(reply_num_list))
    print('첨부이미지URL 리스트 길이 : ', len(img_list))
    print('내용 리스트 길이 : ', len(comment_list))

    # 리스트에 담기
    crawled_list = [date_list, rare_number_list, check_reply_list, nickname_list, 
            writer_url_list, like_num_list, link_num_list, reply_num_list,
            img_list, comment_list]

    return crawled_list

## 현재 파일 위치 확인

In [886]:
# 현재 파일 위치 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/incheon'

## 수집한 파일 저장 폴더 생성

In [887]:
# 수집한 파일을 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/naver_blog'
os.makedirs(crawled_folder_path, exist_ok=True)

## 크롬 드라이버 버전 확인 및 설치

In [888]:
# 크롬 드라이버 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'100'

In [889]:
# 크롬 드라이버 설치 확인
try:
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')
except:
    ca.install(True)
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3261653714.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')


## 페이지 접속

In [890]:
# 페이지 접속
url = 'https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222657505998&categoryNo=9&parentCategoryNo=9&from=thumbnailList'
driver.get(url)
driver.implicitly_wait(3)
driver.maximize_window()

In [891]:
# page 정보
page_info = url.split('=')[2].split('&')[0]
page_info

'222657505998'

## 페이지 스크래핑

### 댓글 버튼 클릭

In [892]:
# 댓글 버튼 클릭
comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')
comment_button.click()
time.sleep(1)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3197372586.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')


### 이전 페이지 이동

In [893]:
# 이전 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/2309467251.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()


In [894]:
list = [[], [], [], [], [], [], [], [], [], []]
len(list), list

(10, [[], [], [], [], [], [], [], [], [], []])

### 1페이지

In [895]:
# 1 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/2965464108.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()


50

In [896]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  50
고유번호 리스트 길이 :  50
답글구분 리스트 길이 :  50
닉네임 리스트 길이 :  50
작성자URL 리스트 길이 :  50
공감수 리스트 길이 :  50
링크수 리스트 길이 :  50
답글수 리스트 길이 :  50
첨부이미지URL 리스트 길이 :  50
내용 리스트 길이 :  50


In [897]:
len(list)

10

In [898]:
len(list[0])

50

### 2페이지

In [899]:
# 2 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[1]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/1995233836.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[1]').click()


50

In [900]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  100
고유번호 리스트 길이 :  100
답글구분 리스트 길이 :  100
닉네임 리스트 길이 :  100
작성자URL 리스트 길이 :  100
공감수 리스트 길이 :  100
링크수 리스트 길이 :  100
답글수 리스트 길이 :  100
첨부이미지URL 리스트 길이 :  100
내용 리스트 길이 :  100


### 3페이지

In [901]:
# 3 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/190575342.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]').click()


50

In [902]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  150
고유번호 리스트 길이 :  150
답글구분 리스트 길이 :  150
닉네임 리스트 길이 :  150
작성자URL 리스트 길이 :  150
공감수 리스트 길이 :  150
링크수 리스트 길이 :  150
답글수 리스트 길이 :  150
첨부이미지URL 리스트 길이 :  150
내용 리스트 길이 :  150


### 4페이지

In [903]:
# 4 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/960497698.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]').click()


50

In [904]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  200
고유번호 리스트 길이 :  200
답글구분 리스트 길이 :  200
닉네임 리스트 길이 :  200
작성자URL 리스트 길이 :  200
공감수 리스트 길이 :  200
링크수 리스트 길이 :  200
답글수 리스트 길이 :  200
첨부이미지URL 리스트 길이 :  200
내용 리스트 길이 :  200


### 5페이지

In [905]:
# 5 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3336210854.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]').click()


50

In [906]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  250
고유번호 리스트 길이 :  250
답글구분 리스트 길이 :  250
닉네임 리스트 길이 :  250
작성자URL 리스트 길이 :  250
공감수 리스트 길이 :  250
링크수 리스트 길이 :  250
답글수 리스트 길이 :  250
첨부이미지URL 리스트 길이 :  250
내용 리스트 길이 :  250


### 6페이지

In [907]:
# 6 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3755473283.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]').click()


50

In [908]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  300
고유번호 리스트 길이 :  300
답글구분 리스트 길이 :  300
닉네임 리스트 길이 :  300
작성자URL 리스트 길이 :  300
공감수 리스트 길이 :  300
링크수 리스트 길이 :  300
답글수 리스트 길이 :  300
첨부이미지URL 리스트 길이 :  300
내용 리스트 길이 :  300


### 7페이지

In [909]:
# 7 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[7]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/1700268311.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[7]').click()


51

In [910]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  351
고유번호 리스트 길이 :  351
답글구분 리스트 길이 :  351
닉네임 리스트 길이 :  351
작성자URL 리스트 길이 :  351
공감수 리스트 길이 :  351
링크수 리스트 길이 :  351
답글수 리스트 길이 :  351
첨부이미지URL 리스트 길이 :  351
내용 리스트 길이 :  351


### 8페이지

In [911]:
# 8 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[8]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/4232212690.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[8]').click()


50

In [912]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  401
고유번호 리스트 길이 :  401
답글구분 리스트 길이 :  401
닉네임 리스트 길이 :  401
작성자URL 리스트 길이 :  401
공감수 리스트 길이 :  401
링크수 리스트 길이 :  401
답글수 리스트 길이 :  401
첨부이미지URL 리스트 길이 :  401
내용 리스트 길이 :  401


### 9페이지

In [913]:
# 9 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[9]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3161500032.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[9]').click()


50

In [914]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  451
고유번호 리스트 길이 :  451
답글구분 리스트 길이 :  451
닉네임 리스트 길이 :  451
작성자URL 리스트 길이 :  451
공감수 리스트 길이 :  451
링크수 리스트 길이 :  451
답글수 리스트 길이 :  451
첨부이미지URL 리스트 길이 :  451
내용 리스트 길이 :  451


### 10페이지

In [915]:
# 10 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[10]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3994551413.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[10]').click()


50

In [916]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  501
고유번호 리스트 길이 :  501
답글구분 리스트 길이 :  501
닉네임 리스트 길이 :  501
작성자URL 리스트 길이 :  501
공감수 리스트 길이 :  501
링크수 리스트 길이 :  501
답글수 리스트 길이 :  501
첨부이미지URL 리스트 길이 :  501
내용 리스트 길이 :  501


### 11페이지

In [917]:
# 다음(11)페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[11]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/711338811.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[11]').click()


50

In [918]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  551
고유번호 리스트 길이 :  551
답글구분 리스트 길이 :  551
닉네임 리스트 길이 :  551
작성자URL 리스트 길이 :  551
공감수 리스트 길이 :  551
링크수 리스트 길이 :  551
답글수 리스트 길이 :  551
첨부이미지URL 리스트 길이 :  551
내용 리스트 길이 :  551


### 12페이지

In [919]:
# 12 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/4264318083.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]').click()


50

In [920]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  601
고유번호 리스트 길이 :  601
답글구분 리스트 길이 :  601
닉네임 리스트 길이 :  601
작성자URL 리스트 길이 :  601
공감수 리스트 길이 :  601
링크수 리스트 길이 :  601
답글수 리스트 길이 :  601
첨부이미지URL 리스트 길이 :  601
내용 리스트 길이 :  601


### 13페이지

In [921]:
# 13 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3413347789.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]').click()


50

In [922]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  651
고유번호 리스트 길이 :  651
답글구분 리스트 길이 :  651
닉네임 리스트 길이 :  651
작성자URL 리스트 길이 :  651
공감수 리스트 길이 :  651
링크수 리스트 길이 :  651
답글수 리스트 길이 :  651
첨부이미지URL 리스트 길이 :  651
내용 리스트 길이 :  651


### 14페이지

In [923]:
# 14 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/91115677.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]').click()


50

In [924]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  701
고유번호 리스트 길이 :  701
답글구분 리스트 길이 :  701
닉네임 리스트 길이 :  701
작성자URL 리스트 길이 :  701
공감수 리스트 길이 :  701
링크수 리스트 길이 :  701
답글수 리스트 길이 :  701
첨부이미지URL 리스트 길이 :  701
내용 리스트 길이 :  701


### 15페이지

In [925]:
# 15 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]').click()
time.sleep( random.uniform(1, 1.5) )

# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3080943050.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]').click()


30

In [926]:
list = crawling(comment_areas, list)

작성일 리스트 길이 :  731
고유번호 리스트 길이 :  731
답글구분 리스트 길이 :  731
닉네임 리스트 길이 :  731
작성자URL 리스트 길이 :  731
공감수 리스트 길이 :  731
링크수 리스트 길이 :  731
답글수 리스트 길이 :  731
첨부이미지URL 리스트 길이 :  731
내용 리스트 길이 :  731


## 수집한 데이터 확인

In [927]:
print('작성일 리스트 길이 : ', len(list[0]))
print('고유번호 리스트 길이 : ', len(list[1]))
print('답글구분 리스트 길이 : ', len(list[2]))
print('닉네임 리스트 길이 : ', len(list[3]))
print('작성자URL 리스트 길이 : ', len(list[4]))
print('공감수 리스트 길이 : ', len(list[5]))
print('링크수 리스트 길이 : ', len(list[6]))
print('답글수 리스트 길이 : ', len(list[7]))
print('첨부이미지URL 리스트 길이 : ', len(list[8]))
print('내용 리스트 길이 : ', len(list[9]))

작성일 리스트 길이 :  731
고유번호 리스트 길이 :  731
답글구분 리스트 길이 :  731
닉네임 리스트 길이 :  731
작성자URL 리스트 길이 :  731
공감수 리스트 길이 :  731
링크수 리스트 길이 :  731
답글수 리스트 길이 :  731
첨부이미지URL 리스트 길이 :  731
내용 리스트 길이 :  731


## df 만들기

In [928]:
dict = {'작성일':list[0], '고유번호':list[1], 
        '답글구분':list[2], '닉네임':list[3], 
        '작성자URL':list[4], '공감수':list[5], 
        '링크수':list[6], '답글수':list[7], 
        '첨부이미지URL':list[8], '내용':list[9]}
df = pd.DataFrame(dict)

In [929]:
df.head()

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
0,2022-02-25 12:58:42,750627914109682094,NaN,문득,https://blog.naver.com/ekdlskdl,0,0,0,NaN,*정답: 3번 @해피쏭
1,2022-02-25 12:59:29,750627965884170523,NaN,푸른하늘현,https://blog.naver.com/blueday118,0,1,0,NaN,정답 : 3번 https://blog.naver.com/blueday118/2226...
2,2022-02-25 13:05:52,750628376070324774,NaN,란,https://blog.naver.com/cdism,0,1,0,NaN,*정답: 3번 https://blog.naver.com/cdism @란 @cdism
3,2022-02-25 13:07:04,750628453866274870,NaN,saypark3824,https://blog.naver.com/saypark3824,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjVfNTkg/MD...,정답 : 3번 입니다 @최준원 학생기자
4,2022-02-25 13:07:35,750628486162415983,NaN,리체,https://blog.naver.com/litvice,0,0,0,NaN,3번~~~^^


In [946]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       731 non-null    object
 1   고유번호      731 non-null    object
 2   답글구분      48 non-null     object
 3   닉네임       731 non-null    object
 4   작성자URL    731 non-null    object
 5   공감수       731 non-null    int64 
 6   링크수       731 non-null    int64 
 7   답글수       731 non-null    int64 
 8   첨부이미지URL  123 non-null    object
 9   내용        731 non-null    object
dtypes: int64(3), object(7)
memory usage: 57.2+ KB


### df 확인
* 고유번호 : undefined 제거
* 닉네임 : 비밀댓글 제거
* 답글 구분 : 비밀댓글 nan값으로 교체

#### 고유번호에서 undefined 제거

In [991]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       731 non-null    object
 1   고유번호      731 non-null    object
 2   답글구분      48 non-null     object
 3   닉네임       731 non-null    object
 4   작성자URL    731 non-null    object
 5   공감수       731 non-null    int64 
 6   링크수       731 non-null    int64 
 7   답글수       731 non-null    int64 
 8   첨부이미지URL  123 non-null    object
 9   내용        731 non-null    object
dtypes: int64(3), object(7)
memory usage: 57.2+ KB


In [992]:
df[df['고유번호'] == 'undefined']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
300,2022-02-27 17:29:16,undefined,NaN,란나,https://blog.naver.com/kiboda,0,0,3,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,


In [993]:
undefined_index = df[df['고유번호'] == 'undefined'].index
undefined_index

Int64Index([300], dtype='int64')

In [994]:
df.iloc[299:304, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
299,2022-02-27 17:29:02,750830872990580750,NaN,란나,https://blog.naver.com/kiboda,0,1,0,NaN,정답: 3번인천광역시의회 항상 응원합니다♡ @사랑님 https://m.blog....
300,2022-02-27 17:29:16,undefined,NaN,란나,https://blog.naver.com/kiboda,0,0,3,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
301,2022-02-27 17:29:16,750830888173961351,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
302,2022-02-27 17:29:28,750830900253556981,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTQ3/MD...,
303,2022-02-27 17:29:38,750830912618365258,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfNDUg/MD...,


In [1001]:
df2 = df.drop(index=undefined_index).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       730 non-null    object
 1   고유번호      730 non-null    object
 2   답글구분      48 non-null     object
 3   닉네임       730 non-null    object
 4   작성자URL    730 non-null    object
 5   공감수       730 non-null    int64 
 6   링크수       730 non-null    int64 
 7   답글수       730 non-null    int64 
 8   첨부이미지URL  122 non-null    object
 9   내용        730 non-null    object
dtypes: int64(3), object(7)
memory usage: 57.2+ KB


In [1002]:
df2.iloc[299:304, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
299,2022-02-27 17:29:02,750830872990580750,NaN,란나,https://blog.naver.com/kiboda,0,1,0,NaN,정답: 3번인천광역시의회 항상 응원합니다♡ @사랑님 https://m.blog....
300,2022-02-27 17:29:16,750830888173961351,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
301,2022-02-27 17:29:28,750830900253556981,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTQ3/MD...,
302,2022-02-27 17:29:38,750830912618365258,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfNDUg/MD...,
303,2022-02-27 17:31:13,750831012895785085,NaN,lhj123400,https://blog.naver.com/lhj123400,0,1,0,NaN,정답 : 3번 https://blog.naver.com/lhj123400/22265...


#### 닉네임에서 비밀댓글 제거

In [1003]:
df2[df2['닉네임'] == '비밀댓글']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
91,2022-02-25 15:34:45,750637968544236341,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
124,2022-02-25 17:56:26,750647096389927242,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
131,2022-02-25 19:45:54,750654147266609902,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
140,2022-02-25 20:32:59,750657180503376843,비밀댓글,비밀댓글,비밀댓글,0,0,1,NaN,비밀댓글
141,2022-02-25 20:33:03,750657185435877425,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
156,2022-02-25 21:33:34,750661084628648583,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
207,2022-02-26 12:01:33,750717003223793952,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
309,2022-02-27 20:39:40,750843155405865456,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
422,2022-03-02 10:41:01,751082900396441610,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
635,2022-03-04 21:37:52,751310761766158964,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글


In [1004]:
secret_reply_index = df2[df2['닉네임'] == '비밀댓글'].index
secret_reply_index

Int64Index([ 91, 124, 131, 140, 141, 156, 207, 309, 422, 635, 636, 637, 638,
            639, 646, 692, 696, 729],
           dtype='int64')

In [1017]:
df3 = df2.drop(index=secret_reply_index).reset_index(drop=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       712 non-null    object
 1   고유번호      712 non-null    object
 2   답글구분      30 non-null     object
 3   닉네임       712 non-null    object
 4   작성자URL    712 non-null    object
 5   공감수       712 non-null    int64 
 6   링크수       712 non-null    int64 
 7   답글수       712 non-null    int64 
 8   첨부이미지URL  122 non-null    object
 9   내용        712 non-null    object
dtypes: int64(3), object(7)
memory usage: 55.8+ KB


In [1018]:
df3['답글구분'].unique()

array([nan, '답글', '비밀댓글'], dtype=object)

#### 답글구분에서 비밀댓글 nan 값으로 교체

In [1019]:
df3[df3['답글구분'] == '비밀댓글']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
625,2022-03-04 21:24:40,751309911094198722,비밀댓글,단단해진나,https://blog.naver.com/snowdropsun,0,4,5,NaN,정답 : 3번 https://m.blog.naver.com/snowdropsun/2...
679,2022-03-04 23:12:05,751316830286512183,비밀댓글,금덩이,https://blog.naver.com/pinkangel9999,0,4,1,NaN,3번인천광역시의회 멋진 정책 법안을 많이 만들어 주셔서 시간가는 줄 모르고 재미나게...


In [1020]:
index3 = df3[df3['답글구분'] == '비밀댓글'].index
index3

Int64Index([625, 679], dtype='int64')

In [1021]:
df4 = df3.replace( {'답글구분':'비밀댓글'}, nan )

In [1022]:
df4.iloc[625:627, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
625,2022-03-04 21:24:40,751309911094198722,NaN,단단해진나,https://blog.naver.com/snowdropsun,0,4,5,NaN,정답 : 3번 https://m.blog.naver.com/snowdropsun/2...
626,2022-03-04 21:25:20,751309953356005600,NaN,행복한여린이,https://blog.naver.com/shiranuimai2,0,1,0,NaN,정답: 3번입니다!! https://blog.naver.com/shiranuimai...


In [1023]:
df4.iloc[678:680, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
678,2022-03-04 23:11:09,751316770945499247,NaN,달빛림,https://blog.naver.com/outside1419,0,0,0,NaN,"<정답: 3번 > 공감, 기존이웃! #인천광역시의 퀴즈 함께 풀어요! @내사랑만두야"
679,2022-03-04 23:12:05,751316830286512183,NaN,금덩이,https://blog.naver.com/pinkangel9999,0,4,1,NaN,3번인천광역시의회 멋진 정책 법안을 많이 만들어 주셔서 시간가는 줄 모르고 재미나게...


In [1024]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       712 non-null    object
 1   고유번호      712 non-null    object
 2   답글구분      28 non-null     object
 3   닉네임       712 non-null    object
 4   작성자URL    712 non-null    object
 5   공감수       712 non-null    int64 
 6   링크수       712 non-null    int64 
 7   답글수       712 non-null    int64 
 8   첨부이미지URL  122 non-null    object
 9   내용        712 non-null    object
dtypes: int64(3), object(7)
memory usage: 55.8+ KB


In [1013]:
final_df = df4.copy()

## 파일로 출력

In [1014]:
crawled_folder_path

'c:/data/Wantreez/Crawling/incheon/crawled_data/naver_blog'

In [1015]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
today_date

'20220418_095347'

In [1016]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
file_name = f'naver_blog_{today_date}.xlsx'
final_df.to_excel(crawled_folder_path + '/' + file_name, index=False, encoding='utf-8')
print(f'{file_name} 출력완료')

naver_blog_20220418_095349.xlsx 출력완료


# 코드 마지막